In [3]:
import cv2
import pickle
import numpy as np
import json
import os
from glob import glob
import torch
from PIL import Image
from torch import nn
from torchvision import transforms
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch import optim
from tqdm.auto import tqdm
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sys import getsizeof
from pympler import asizeof
import pandas as pd
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [1]:
import cv2
import mediapipe as mp
import numpy as np
# 카메라 캡처를 초기화합니다.
cap = cv2.VideoCapture(0)  # 0은 기본 카메라를 의미합니다.
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
# 녹화 상태를 나타내는 변수를 초기화합니다.
recording = False
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 비디오 코덱을 설정합니다.
out = None

while cap.isOpened():
    
    with mp_hands.Hands(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
        # 카메라에서 프레임을 읽어옵니다.
        ret, frame = cap.read()
        success, image = cap.read()
        frame = cv2.flip(frame, 1)
    #     # 프레임을 화면에 표시합니다.
        cv2.imshow('Video', frame)

        # 's' 키를 누르면 녹화를 시작합니다.
        if cv2.waitKey(1) & 0xFF == ord('s'):
            recording = True
            out = cv2.VideoWriter('output.avi', fourcc, 20.0, (frame.shape[1], frame.shape[0]))  # 출력 파일을 설정합니다.

        # 녹화 중일 때 프레임을 저장합니다.
        if recording:
            out.write(frame)

        # 't' 키를 누르면 녹화를 중지하고 저장 파일을 닫습니다.
        if cv2.waitKey(1) & 0xFF == ord('t'):
            if recording:
                recording = False
                out.release()  # 녹화 파일을 닫습니다.
                print("녹화가 종료되었습니다.")
        
        # 'q' 키를 누르면 루프를 종료합니다.
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# 카메라 캡처를 해제하고 창을 닫습니다.
cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = None
recording = False

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        continue

    # 화면에 손 감지된 랜드마크 표시
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    if results.multi_hand_landmarks:
        for landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)
            
    frame = cv2.flip(frame, 1)
    cv2.imshow('Video', frame)

    # 녹화 시작
    if cv2.waitKey(1) & 0xFF == ord('s'):
        recording = True
        out = cv2.VideoWriter('output.avi', fourcc, 20.0, (frame.shape[1], frame.shape[0]))

    # 녹화 중지 및 파일 저장
    if cv2.waitKey(1) & 0xFF == ord('t'):
        if recording:
            recording = False
            out.release()
            print("녹화가 종료되었습니다.")

    # 녹화 중일 때 프레임 저장
    if recording:
        out.write(frame)

    # 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [7]:
import cv2
import mediapipe as mp
import numpy as np

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = None
recording = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    # 손 감지 및 랜드마크 추출
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 손 랜드마크만 추출하여 이진 이미지 생성 (손의 랜드마크만 흰색으로, 배경은 검은색으로)
            hand_landmarks_image = np.zeros_like(frame)
            for lm in hand_landmarks.landmark:
                x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                cv2.circle(hand_landmarks_image, (x, y), 5, (255, 255, 255), -1)

    cv2.imshow('Video', frame)

    # 녹화 시작
    if cv2.waitKey(1) & 0xFF == ord('s'):
        recording = True
        out = cv2.VideoWriter('output.avi', fourcc, 20.0, (frame.shape[1], frame.shape[0]))

    # 녹화 중지 및 파일 저장
    if cv2.waitKey(1) & 0xFF == ord('t'):
        if recording:
            recording = False
            out.release()
            print("녹화가 종료되었습니다.")

    # 녹화 중일 때 프레임 저장
    if recording:
        out.write(hand_landmarks_image)

    # 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [15]:
import cv2
import mediapipe as mp
import numpy as np

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = None
recording = False

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    if not ret:
        continue

    # 손 감지 및 랜드마크 추출
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 손 랜드마크만 추출하여 이진 이미지 생성 (손의 랜드마크만 흰색으로, 배경은 검은색으로)
            hand_landmarks_image = np.zeros_like(frame)
            for lm in hand_landmarks.landmark:
                x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                cv2.circle(hand_landmarks_image, (x, y), 5, (255, 255, 255), -1)

            # 랜드마크 간의 선 그리기
            connections = mp_hands.HAND_CONNECTIONS
            for connection in connections:
                connection_point_a = tuple(
                    np.multiply([hand_landmarks.landmark[connection[0]].x, hand_landmarks.landmark[connection[0]].y],
                                [frame.shape[1], frame.shape[0]]).astype(int))
                connection_point_b = tuple(
                    np.multiply([hand_landmarks.landmark[connection[1]].x, hand_landmarks.landmark[connection[1]].y],
                                [frame.shape[1], frame.shape[0]]).astype(int))
                cv2.line(hand_landmarks_image, connection_point_a, connection_point_b, (255, 255, 255), 2)
    
    cv2.imshow('Video', hand_landmarks_image)

    # 녹화 시작
    if cv2.waitKey(1) & 0xFF == ord('s'):
        recording = True
        out = cv2.VideoWriter('output.avi', fourcc, 20.0, (frame.shape[1], frame.shape[0]))

    # 녹화 중지 및 파일 저장
    if cv2.waitKey(1) & 0xFF == ord('t'):
        if recording:
            recording = False
            out.release()
            print("녹화가 종료되었습니다.")

    # 녹화 중일 때 프레임 저장
    if recording:
        out.write(hand_landmarks_image)

    # 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
